## Here we compare models of different depth and architecture on clear data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
mpl.use("TkAgg")

from sklearn.preprocessing import normalize#, MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import h5py
import mne
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Reshape, GRU, LSTM, Lambda, RepeatVector, Reshape, Dropout, Conv1D, UpSampling1D, Bidirectional

from os import walk, listdir


%matplotlib inline
data_path = "../../data/train/"
clear_data_path = "/media/valbub/Docs/data/train/"
raw_data_path = "../../data/resting_state/"

Using TensorFlow backend.


In [2]:
class AutoEncoder(object):
    def __init__(self, 
             input_dim = (5, 58), 
             encoded_dim = (1, 58), 
             loss="mse", 
             optimizer="adadelta", 
             activation=("relu", "sigmoid", "tanh", "elu"),
             act_idx=(0, 0),
             kernel = 5):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim
            
            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (np.array(X) - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()
    
    def prepare_clear_data(self, data_path, limit=2):
        train_eeg_dir = data_path
        train_eeg_names = [x for x in listdir(train_eeg_dir) 
                         if x[-3:] == ".h5"]
        data = np.zeros((0, self.input_dim[0], self.input_dim[1]))

        flag = 0
        err_files = 0
        for eeg_name in train_eeg_names:
            if flag == limit:
                break
            flag += 1
            h5_file = h5py.File(train_eeg_dir + eeg_name, 'r')
            a_group_key = list(h5_file.keys())[0]
            eeg_data = np.array(h5_file[a_group_key]).T
            batches = np.array(self._getBatches(eeg_data, batch_size=self.input_dim[0]))
            data = np.concatenate((data, batches), axis=0)
        return data
    
    def prepare_raw_data(self, data_path, limit=2):
        def preparefile(file_path):
            raw = np.array(mne.io.read_raw_brainvision(file_path, preload=True).to_data_frame())
            batches = np.array(self._getBatches(raw, batch_size=self.input_dim[0]))
            del raw
            return batches
        files = []
        data = []
        for elem in walk(data_path):
            for file in elem[-1]:
                if file[-4:] == "vhdr":
                    files.append(file)
        data = np.ndarray(shape=(0, self.input_dim[0], self.input_dim[1]))
        flag = 0
        for file in files:
            flag += 1
            file_name = data_path + file
            if flag == limit:
                break
            batches =  preparefile(file_name)
            data = np.concatenate((data, batches), axis=0)
        return data
    
    def fit(self, X_train, epochs=50):
        X_scaled = self.scaler.fit_transform(X_train)
        return self.autoencoder.fit(X_scaled, X_scaled, epochs = epochs)
    
    def fit_scaler(self, X_train):
        self.scaler.fit_transform(X_train)
    
    def encode(self, df):
        return self._predict(df, self.encoder, self.input_dim[0])
    
    def decode(self, df):
        return self._predict(df, self.decoder, self.encoded_dim[1])
    
    def run(self, df):
        return self._predict(df, self.autoencoder, self.input_dim[0])
    
    def save(self, path, part="autoencoder"):
        if part == "encoder":
            self.encoder.save(path)
        elif part == "decoder":
            self.decoder.save(path)
        elif part == "autoencoder":
            self.autoencoder.save(path)
        pass
    
    def load(self, path, part="autoencoder", X_train=None):
        if part == "encoder":
            self.encoder = keras.models.load_model(path)
        elif part == "decoder":
            self.decoder = keras.models.load_model(path)
        elif part == "autoencoder":
            self.autoencoder = keras.models.load_model(path)
        if x_train is not None:
            self.fit_scaler(X_train)
    

    def _predict(self, df, model):
        batches = self.scaler.transform(df)
        batches = tuple(self._predictBatch(batch.reshape((1, *batch.shape)), model) for batch in batches)
        batches = self._concatBatches(batches) 
        return self.scaler.reverse_transform(batches)
    
    def _predictBatch(self, batch, model):
        return model.predict(batch)
    
    def _getBatches(self, arr, batch_size, axis=0):
        n_batches = arr.shape[axis] // batch_size
        result = np.array_split(arr, n_batches, axis=axis)
        i = 0
        while result[i].shape[0] != batch_size:
            i += 1
        result = result[i:]
        return result
    
    def _concatBatches(self, batches, axis=0):
        return np.concatenate(batches, axis=axis)


In [3]:
class AU_Stage_1(AutoEncoder):
        def __init__(self, 
                 input_dim = (5, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 5):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._flat = Flatten()(self._inputs)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._flat)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=(encoded_dim[0]*encoded_dim[1],))
            self._flat_decoded = Dense(input_dim[0]*input_dim[1], activation=activation[1])(self._encoded_inputs)
            self._decoded = Reshape(input_dim)(self._flat_decoded)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))
            
            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (np.array(X) - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [4]:
class AU_Stage_2(AutoEncoder):
        def __init__(self, 
                 input_dim = (5, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 5):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._conv = Conv1D(filters=encoded_dim[1], kernel_size=kernel)(self._inputs) 
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._conv)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._decoded = Reshape(input_dim)(self._flat_decoded)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)
            
            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()


In [5]:
class AU_Stage_3(AutoEncoder):
        def __init__(self, 
                 input_dim = (5, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 5,
                 folds=100):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._lambda = Lambda(lambda x: K.round(x * folds) / folds)(self._inputs)
            self._lstm = LSTM(input_dim[1], return_sequences=True, dropout=0, recurrent_dropout=0.1)(self._lambda)
            self._conv = Conv1D(filters=encoded_dim[1], kernel_size=input_dim[0])(self._lstm)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._conv)
            self._encoded = Reshape(encoded_dim)(self._dense)
            
            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded_1 = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._flat_decoded_2 = UpSampling1D(size=input_dim[0] // encoded_dim[0])(self._flat_decoded_1)
            self._lstm_2 = LSTM(input_dim[1], return_sequences=True)(self._flat_decoded_2)
            self._decoded = Reshape(input_dim)(self._lstm_2)
            
            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))
            
            self.autoencoder.compile(optimizer=optimizer, loss=loss)
            
            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))  + 1e-10
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [6]:
class AU_Stage_3_5(AutoEncoder):
        def __init__(self, 
                 input_dim = (5, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 5):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._lstm = GRU(input_dim[1], return_sequences=True, dropout=0, recurrent_dropout=0.1)(self._inputs)
            self._conv = Conv1D(filters=encoded_dim[1], kernel_size=input_dim[0])(self._lstm)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._conv)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded_1 = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._flat_decoded_2 = UpSampling1D(size=input_dim[0] // encoded_dim[0])(self._flat_decoded_1)
            self._lstm_2 = GRU(input_dim[1], return_sequences=True)(self._flat_decoded_2)
            self._decoded = Reshape(input_dim)(self._lstm_2)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1)) + 1e-10
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [7]:
class AU_Stage_4(AutoEncoder):
        def __init__(self, 
                 input_dim = (5, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 5,
                 folds = 100):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._lambda = Lambda(lambda x: K.round(x * folds) / folds)(self._inputs)
            self._lstm = LSTM(input_dim[1], return_sequences=True, dropout=0, recurrent_dropout=0.1)(self._lambda)
            self._reshape = Reshape((-1, input_dim[1] * input_dim[0]))(self._lstm)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._reshape)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded_1 = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._lstm_2 = LSTM(input_dim[1] * input_dim[0], return_sequences=True)(self._flat_decoded_1)
            self._decoded = Reshape((-1, input_dim[1]))(self._lstm_2)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1)) + 1e-10
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [8]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

epoch_numb = 50
limit = 14

In [9]:
# here we get data for all experiments in future
au_1 = AU_Stage_1()
data_set = au_1.prepare_clear_data(clear_data_path, limit=limit)
train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)
data_set.shape

(1663700, 5, 58)

In [10]:
au_1.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_1/"
au_1.save(folder + "au")
au_1.save(folder + "en")
au_1.save(folder + "de")

Epoch 1/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 4.3283e-04
Epoch 2/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 1.8347e-04
Epoch 3/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 1.3144e-04
Epoch 4/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 1.1293e-04
Epoch 5/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 1.0336e-04
Epoch 6/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 9.6867e-05
Epoch 7/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 9.1089e-05
Epoch 8/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 8.5847e-05
Epoch 9/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 8.1284e-05
Epoch 10/50
1164590/1164590 [==============================] - 46s 39us/step - loss: 7.7352e-05
Epoch 11/50
1164590/1164590 [====================

In [11]:
pr = au_1._predict(test_data, au_1.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print('\nDense')
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense

Clear data
0.8629372410722346
5.079840092974387e-06
5.368622227515531e-09



In [12]:
del au_1

In [13]:
au_2 = AU_Stage_2()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_2.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_2.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_2/"
au_2.save(folder + "au")
au_2.save(folder + "en")
au_2.save(folder + "de")

Epoch 1/50
1164590/1164590 [==============================] - 47s 41us/step - loss: 3.9776e-04
Epoch 2/50
1164590/1164590 [==============================] - 47s 41us/step - loss: 1.5750e-04
Epoch 3/50
1164590/1164590 [==============================] - 47s 41us/step - loss: 1.2377e-04
Epoch 4/50
1164590/1164590 [==============================] - 47s 41us/step - loss: 1.0546e-04
Epoch 5/50
1164590/1164590 [==============================] - 48s 41us/step - loss: 9.6167e-05
Epoch 6/50
1164590/1164590 [==============================] - 48s 41us/step - loss: 8.8723e-05
Epoch 7/50
1164590/1164590 [==============================] - 48s 41us/step - loss: 8.2866e-05
Epoch 8/50
1164590/1164590 [==============================] - 47s 41us/step - loss: 7.8204e-05
Epoch 9/50
1164590/1164590 [==============================] - 48s 41us/step - loss: 7.4097e-05
Epoch 10/50
1164590/1164590 [==============================] - 47s 41us/step - loss: 7.0298e-05
Epoch 11/50
1164590/1164590 [====================

In [14]:
pr = au_2._predict(test_data, au_2.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print('\nDense+Conv')
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+Conv

Clear data
0.8721954383720413
5.014486137700528e-06
6.668714340613226e-09



In [15]:
del au_2

In [16]:
au_3 = AU_Stage_3()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_3.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_3.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_3/"
au_3.save(folder + "au")
au_3.save(folder + "en")
au_3.save(folder + "de")

Epoch 1/50
1164590/1164590 [==============================] - 214s 183us/step - loss: 5.5478e-04
Epoch 2/50
1164590/1164590 [==============================] - 213s 183us/step - loss: 2.3129e-04
Epoch 3/50
1164590/1164590 [==============================] - 216s 185us/step - loss: 1.5157e-04
Epoch 4/50
1164590/1164590 [==============================] - 216s 186us/step - loss: 1.3814e-04
Epoch 5/50
1164590/1164590 [==============================] - 216s 185us/step - loss: 1.3555e-04
Epoch 6/50
1164590/1164590 [==============================] - 215s 185us/step - loss: 1.3338e-04
Epoch 7/50
1164590/1164590 [==============================] - 215s 185us/step - loss: 1.3034e-04
Epoch 8/50
1164590/1164590 [==============================] - 216s 185us/step - loss: 1.2434e-04
Epoch 9/50
1164590/1164590 [==============================] - 216s 186us/step - loss: 1.1747e-04
Epoch 10/50
1164590/1164590 [==============================] - 216s 186us/step - loss: 1.1148e-04
Epoch 11/50
1164590/1164590 [

In [17]:
pr = au_3._predict(test_data, au_3.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print("\nDense+Conv+LSTM")
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+Conv+LSTM

Clear data
0.7572858637508183
9.641932782303063e-06
1.3170053543019765e-08



In [18]:
del au_3

In [19]:
au_3_5 = AU_Stage_3_5()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_3_5.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_3_5.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_3_5/"
au_3_5.save(folder + "au")
au_3_5.save(folder + "en")
au_3_5.save(folder + "de")

Epoch 1/50
1164590/1164590 [==============================] - 186s 160us/step - loss: 3.2270e-04
Epoch 2/50
1164590/1164590 [==============================] - 186s 160us/step - loss: 1.2192e-04
Epoch 3/50
1164590/1164590 [==============================] - 187s 160us/step - loss: 1.0239e-04
Epoch 4/50
1164590/1164590 [==============================] - 187s 160us/step - loss: 9.1380e-05
Epoch 5/50
1164590/1164590 [==============================] - 189s 163us/step - loss: 8.3100e-05
Epoch 6/50
1164590/1164590 [==============================] - 189s 162us/step - loss: 7.7032e-05
Epoch 7/50
1164590/1164590 [==============================] - 189s 163us/step - loss: 7.2240e-05
Epoch 8/50
1164590/1164590 [==============================] - 190s 163us/step - loss: 6.9036e-05
Epoch 9/50
1164590/1164590 [==============================] - 190s 163us/step - loss: 6.6678e-05
Epoch 10/50
1164590/1164590 [==============================] - 189s 163us/step - loss: 6.4504e-05
Epoch 11/50
1164590/1164590 [

In [20]:
pr = au_3_5._predict(test_data, au_3_5.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print("\nDense+Conv+GRU")
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+Conv+GRU

Clear data
0.8661866968120966
5.41238914054588e-06
1.3698147217852704e-09



In [21]:
del au_3_5

In [22]:
au_4 = AU_Stage_4()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_4.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_4.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_4/"
au_4.save(folder + "au")
au_4.save(folder + "en")
au_4.save(folder + "de")

Epoch 1/50
1164590/1164590 [==============================] - 384s 329us/step - loss: 6.6335e-04
Epoch 2/50
1164590/1164590 [==============================] - 383s 329us/step - loss: 2.8707e-04
Epoch 3/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 2.6696e-04
Epoch 4/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 2.3863e-04
Epoch 5/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 1.6729e-04
Epoch 6/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 1.5943e-04
Epoch 7/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 1.5489e-04
Epoch 8/50
1164590/1164590 [==============================] - 385s 330us/step - loss: 1.4149e-04
Epoch 9/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 1.1903e-04
Epoch 10/50
1164590/1164590 [==============================] - 384s 330us/step - loss: 1.1265e-04
Epoch 11/50
1164590/1164590 [

In [23]:
pr = au_4._predict(test_data, au_4.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print("\nDense+LSTM")
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+LSTM

Clear data
0.7932128982892078
4.4900777431201955e-06
1.0678865418996525e-08



In [24]:
del au_4